# Scrape users using selenium with PhantomJS

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from scrapy.selector import Selector
import re
import time
import csv
import os
import pymongo
from pymongo import InsertOne
from tqdm import tqdm
from itertools import islice

def read_gid_link_numratings(start, stop):
    gids = []
    glinks = []
    numratings = []
    with open('csv/numratings', 'r') as f:
        reader = csv.reader(islice(f, start, stop+1))
        for row in reader:
            gids.append(row[0])
            glinks.append(row[1])
            numratings.append(row[2])
    return gids, glinks, numratings

def set_max_pages(numrating):
    numrating = int(numrating)
    if numrating%50 != 0:
        max_pages = numrating//50 + 1
    else:
        max_pages = int(numrating/50)
    return max_pages

def write_rating_csv(user_names, gid, ratings, curr_page):
    with open('csv/users/{}_{}'.format(gid, curr_page), 'a') as f:
        for i, user in enumerate(user_names):
            f.write('{},{},{}\n'.format(user, gid, ratings[i]))
            
            
def check_page_loaded(driver, last_page_el):
    first_rater = []
    page_el = []
    while not first_rater or not page_el:
        try:
            first_rater = driver.find_element_by_xpath("//ratings-module//li[@class='summary-item summary-rating-item']\
                                                        [1]/div[@class='comment-header']/div/div/a")
            page_el = first_rater.text
        except:
            time.sleep(.5)
    while page_el == last_page_el:
        try:
            first_rater = driver.find_element_by_xpath("//ratings-module//li[@class='summary-item summary-rating-item']\
                                                        [1]/div[@class='comment-header']/div/div/a")
            page_el = first_rater.text
        except:
            time.sleep(.5)
            
    return page_el

def get_game_raters(gid, glink, max_pages, curr_page=1):

    driver = webdriver.PhantomJS()
    
    last_page_el = []

    while curr_page <= max_pages:
        driver.get('https://boardgamegeek.com{}/ratings?pageid={}&rated=1'.format(glink, curr_page))
        
        last_page_el = check_page_loaded(driver, last_page_el)
        
        html = driver.page_source

        xpath_user_names = "//ratings-module//div[@class='comment-header']/div/div/a/text()"
        xpath_user_links = "//ratings-module//div[@class='comment-header']/div/div/a/@href"
        xpath_ratings = "//ratings-module//li/div[@class='summary-item-callout']/div/text()"

        user_names = Selector(text=html).xpath(xpath_user_names).extract()
        user_links = Selector(text=html).xpath(xpath_user_links).extract()
        dirty_ratings = Selector(text=html).xpath(xpath_ratings).extract()

        ratings = []
        for rating in dirty_ratings:
            ratings.append(re.sub('\s', '', rating))

        write_rating_csv(user_names, gid, ratings, curr_page)

        curr_page += 1
        
    driver.quit()

def get_user_ratings(start, stop):
    gids, glinks, numratings = read_gid_link_numratings(start, stop)
    for gid, glink, numrating in tqdm(zip(gids, glinks, numratings)):
        max_pages = set_max_pages(numrating)
        get_game_raters(gid, glink, max_pages)

In [ ]:
get_user_ratings(27, 75)

0it [00:00, ?it/s]